#Carga de datos


In [1]:
# OASM
#02/11/2024
# DataSet analisis de sentimientos de productos de amazon 1-5
# En el data la pagina obtenida del dataset manejan los datos como
# 1-3 como negativas y 4-5 como positivos
# para tener un balanceo de 50 y 50, ya que actualmente esta 60-40
# podriamos aumentar el numero de comentarios similares en clasifcados como buenos
# esto en caso de que solo clasificquemos como bueno positivo o ngeativo en cawso contraario
#que tomemos cada nivel como una clase entonces no seria necesaria balancear los datos todos estan equilibradas.
# ESTA EN ESPAÑOL E INGLES LAS RESE{ÑAS

import pandas as pd
import matplotlib.pyplot as plt
from matplotlib.colors import LinearSegmentedColormap
import seaborn as sns
import warnings

warnings.filterwarnings('ignore')

data = pd.read_csv('https://raw.githubusercontent.com/ignaciomsarmiento/RecomSystemsLectures/main/L07_sentimientos/data/Amazon.csv', index_col="Unnamed: 0")

data.head()
data['sentiment'] = data['reviews.rating'].apply(lambda x: 'positive' if x >= 4 else 'negative')

#reseñas positivas y negativas
"""sns.countplot(x='sentiment', data=data, palette='coolwarm')
plt.title('Distribucion de Reseñas Positivas y Negativas')
plt.xlabel('Sentimiento')
plt.ylabel('Cantidad')
plt.show()"""

# mapear
puntaje = {
    1 : 0,
    2: 0,
    3: 0,
    4: 1,
    5: 1
}

sentimiento = {
    0: 'NEGATIVO',
    1: 'POSITIVO',
}

# MAPEANDO
data['puntaje'] = data['reviews.rating'].map(puntaje)
data['sentimiento'] = data['puntaje'].map(sentimiento)

data.head()



,id,reviews.text,reviews.rating,reviews.text_esp,sentiment,puntaje,sentimiento
0,AVqVGWQDv8e3D1O-ldFr,This is a very nice tablet for my GF who has n...,5.0,Esta es una tableta muy agradable para mi novi...,positive,1,POSITIVO
1,AVqkIhwDv8e3D1O-lebb,Love this tablet. Easy to use. And price was r...,5.0,Me encanta esta tableta. Fácil de usar. Y el p...,positive,1,POSITIVO
2,AVphgVaX1cnluZ0-DR74,Affordable price awesome quality I love my Ama...,5.0,Precio asequible calidad increíble Me encanta ...,positive,1,POSITIVO
3,AVphgVaX1cnluZ0-DR74,I bought this after speaking with a sales rep ...,5.0,Compré esto después de hablar con un represent...,positive,1,POSITIVO
4,AVqVGWLKnnc1JgDc3jF1,Bought this tablet for my 2 &1/2 year old and ...,5.0,Compré esta tableta para mi hijo de 2 años y m...,positive,1,POSITIVO


#Preprocesamiento


In [2]:
!pip install num2words spacy pandas
!python -m spacy download es_core_news_sm


import string
import spacy
from num2words import num2words

data = data[['reviews.text_esp', 'reviews.rating', 'sentimiento']]

nlp = spacy.load('es_core_news_sm')

stop_words_adicionales = {"¡", "-", "—", "http","<",">"}

for palabra in stop_words_adicionales:
    nlp.Defaults.stop_words.add(palabra)

def procesar_texto(texto):
    oraciones = texto.split('\n')
    total_oraciones = len(oraciones)
    oraciones_tokenizadas = []

    for oracion in oraciones:
        tokens = []
        doc = nlp(oracion)
        for token in doc:
                if token.is_digit:
                    try:
                        numero = int(token.text)  # Convertir texto a número
                        palabra_letras = num2words(numero, lang='es')  # Convertir número a palabras
                        tokens.append(palabra_letras)
                    except ValueError:
                        pass
                else:
                    lemma = token.text.lower()  # Convertir a minúsculas  # Quitar puntuación
                    if lemma and lemma not in stop_words_adicionales:  # Evitar agregar cadenas vacías y stop words
                        tokens.append(lemma)
        oraciones_tokenizadas.append(" ".join(tokens))

    return total_oraciones, oraciones_tokenizadas

data['total_oraciones'], data['oraciones_tokenizadas'] = zip(*data['reviews.text_esp'].apply(procesar_texto))

print(data[['reviews.text_esp', 'oraciones_tokenizadas', 'reviews.rating']].head())


  Preparing metadata (setup.py) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 163.5/163.5 kB 11.8 MB/s eta 0:00:00
  Created wheel for docopt: filename=docopt-0.6.2-py2.py3-none-any.whl size=13706 sha256=45059481aa87753b9b493604035f8a65584a64497ba25643195ef24b3a33d0df
  Stored in directory: /root/.cache/pip/wheels/fc/ab/d4/5da2067ac95b36618c629a5f93f809425700506f72c9732fac
Successfully built docopt
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 12.9/12.9 MB 81.1 MB/s eta 0:00:00
✔ Download and installation successful
You can now load the package via spacy.load('es_core_news_sm')
⚠ Restart to reload dependencies
If you are in a Jupyter or Colab notebook, you may need to restart Python in
order to load all the package's dependencies. You can do this by selecting the
'Restart kernel' or 'Restart runtime' option.
                                    reviews.text_esp  \
0  Esta es una tableta muy agradable para mi novi...   
1  Me encanta esta tableta. Fácil de usar. Y el p...   
2  Pr

#Obtener embeddings

In [3]:
import torch
import numpy as np
from transformers import AutoTokenizer, AutoModel

# Inicializar el modelo BERT
tokenizer = AutoTokenizer.from_pretrained("dccuchile/bert-base-spanish-wwm-cased")
model = AutoModel.from_pretrained("dccuchile/bert-base-spanish-wwm-cased", ignore_mismatched_sizes=True)

# Función para obtener el embedding promedio de una oración
def obtener_embedding_oracion(oracion):
    inputs = tokenizer(oracion, return_tensors="pt", padding=True, truncation=True)
    with torch.no_grad():
        outputs = model(**inputs)
    return outputs.last_hidden_state.mean(dim=1).squeeze().numpy()

# Función para obtener el embedding promedio de una reseña completa
def obtener_embedding_resena(oraciones_tokenizadas):
    embeddings_oraciones = [obtener_embedding_oracion(oracion) for oracion in oraciones_tokenizadas]
    return np.mean(embeddings_oraciones, axis=0) if embeddings_oraciones else np.zeros(model.config.hidden_size)

# Obtener embeddings con etiquetas
data['embedding_resena'] = data['oraciones_tokenizadas'].apply(obtener_embedding_resena)

# Convertir a DataFrame
embeddings_df = pd.DataFrame(data['embedding_resena'].to_list())
embeddings_df['sentimiento'] = data['sentimiento'].values

# Guardar los embeddings con etiquetas en un archivo CSV
embeddings_df.to_csv('embeddings_con_etiqueta.csv', index=False)

print("Embeddings por reseña con etiquetas guardados en 'embeddings_con_etiqueta.csv'.")


tokenizer_config.json:   0%|          | 0.00/364 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/648 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/242k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/480k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/134 [00:00<?, ?B/s]

pytorch_model.bin:   0%|          | 0.00/440M [00:00<?, ?B/s]

Some weights of BertModel were not initialized from the model checkpoint at dccuchile/bert-base-spanish-wwm-cased and are newly initialized: ['bert.pooler.dense.bias', 'bert.pooler.dense.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


Embeddings por reseña con etiquetas guardados en 'embeddings_con_etiqueta.csv'.


# Grid search de modelos para una mejor precision


In [5]:
# 01/ 01/ 2025
# este programa realiza una busqued a de hiperparametros para seleccionar la que
# la que arroje mejor precision esto se hace en paralelo combinando diferentes hiperparametros
# este paso con el fin de elegir los mejores hiperparametros para la clasifciacion posterior a eswte paso
!pip install multiprocess

import itertools
import multiprocess
import time
import numpy as np
from sklearn.metrics import accuracy_score, recall_score
import pandas as pd
from sklearn.svm import SVC
from sklearn.naive_bayes import GaussianNB
from sklearn.neural_network import MLPClassifier
from sklearn.model_selection import train_test_split

n_cores = 4

def Nivelacion_de_Cargas(n_cores, lista_inicial):
    lista_final = []
    longitud_li = len(lista_inicial)
    carga = longitud_li // n_cores
    salidas = longitud_li % n_cores
    contador = 0

    for i in range(n_cores):
        if i < salidas:
            carga2 = contador + carga + 1
        else:
            carga2 = contador + carga
        lista_final.append(lista_inicial[contador:carga2])
        contador = carga2
    return lista_final

# Definir parámetros para SVM
param_grid_svm = {
    'C': [0.1, 1, 10],
    'kernel': ['linear', 'rbf', 'poly'],
    'gamma': ['scale', 0.001, 0.01]
}

keys_svm, values_svm = zip(*param_grid_svm.items())
combinations_svm = [dict(zip(keys_svm, v)) for v in itertools.product(*values_svm)]

# Definir parámetros para MLPClassifier (Red Neuronal)
param_grid_rn = {
    'hidden_layer_sizes': [(50,), (100,)],
    'activation': ['tanh', 'relu'],
    'solver': ['adam', 'sgd']
}

keys_rn, values_rn = zip(*param_grid_rn.items())
combinations_rn = [dict(zip(keys_rn, v)) for v in itertools.product(*values_rn)]

# Evaluar conjunto de hiperparámetros para SVM
def evaluate_svm(hyperparameter_set, mejor_result, lock):
    df = pd.read_csv('embeddings_con_etiqueta.csv')
    X = df.iloc[:, :-1].values
    y = df.iloc[:, -1].values

    X_train, X_test, y_train, y_test = train_test_split(X, y, stratify=y, test_size=0.20, random_state=42)

    for s in hyperparameter_set:
        clf = SVC()
        clf.set_params(C=s['C'], kernel=s['kernel'], gamma=s['gamma'])
        clf.fit(X_train, y_train)

        y_pred = clf.predict(X_test)

        proce_accuracy = accuracy_score(y_test, y_pred)
        proce_recall = recall_score(y_test, y_pred, pos_label="POSITIVO")

        lock.acquire()
        if proce_accuracy > mejor_result['accuracy']:
            mejor_result['accuracy'] = proce_accuracy
            mejor_result['recall'] = proce_recall
            mejor_result['params'] = s
        lock.release()

# Evaluar conjunto de hiperparámetros para MLPClassifier
def evaluate_rn(hyperparameter_set, mejor_result, lock):
    df = pd.read_csv('embeddings_con_etiqueta.csv')
    X = df.iloc[:, :-1].values
    y = df.iloc[:, -1].values

    X_train, X_test, y_train, y_test = train_test_split(X, y, stratify=y, test_size=0.20, random_state=42)

    for s in hyperparameter_set:
        clf = MLPClassifier()
        clf.set_params(hidden_layer_sizes=s['hidden_layer_sizes'], activation=s['activation'], solver=s['solver'])
        clf.fit(X_train, y_train)

        y_pred = clf.predict(X_test)

        proce_accuracy = accuracy_score(y_test, y_pred)
        proce_recall = recall_score(y_test, y_pred, pos_label="POSITIVO")

        lock.acquire()
        if proce_accuracy > mejor_result['accuracy']:
            mejor_result['accuracy'] = proce_accuracy
            mejor_result['recall'] = proce_recall
            mejor_result['params'] = s
        lock.release()


def evaluate_nb(mejor_result, lock):
    df = pd.read_csv('embeddings_con_etiqueta.csv')
    X = df.iloc[:, :-1].values
    y = df.iloc[:, -1].values

    X_train, X_test, y_train, y_test = train_test_split(X, y, stratify=y, test_size=0.20, random_state=42)

    clf = GaussianNB()
    clf.fit(X_train, y_train)

    y_pred = clf.predict(X_test)

    proce_accuracy = accuracy_score(y_test, y_pred)
    proce_recall = recall_score(y_test, y_pred, pos_label="POSITIVO")

    lock.acquire()
    if proce_accuracy > mejor_result['accuracy']:
        mejor_result['accuracy'] = proce_accuracy
        mejor_result['recall'] = proce_recall
        mejor_result['params'] = None
    lock.release()

if __name__ == '__main__':
    threads = []
    lock = multiprocess.Lock()

    with multiprocess.Manager() as manager:
        mejor_result_svm = manager.dict({'accuracy': 0, 'recall': 0, 'params': None})
        mejor_result_rn = manager.dict({'accuracy': 0, 'recall': 0, 'params': None})
        mejor_result_nb = manager.dict({'accuracy': 0, 'recall': 0, 'params': None})

        start_time = time.perf_counter()

        splits_svm = Nivelacion_de_Cargas(n_cores, combinations_svm)
        splits_rn = Nivelacion_de_Cargas(n_cores, combinations_rn)

        # Crear y ejecutar los procesos
        for i in range(n_cores):
            threads.append(multiprocess.Process(target=evaluate_svm, args=(splits_svm[i], mejor_result_svm, lock)))
            threads.append(multiprocess.Process(target=evaluate_rn, args=(splits_rn[i], mejor_result_rn, lock)))

        # Proceso para GaussianNB (sin splits)
        threads.append(multiprocess.Process(target=evaluate_nb, args=(mejor_result_nb, lock)))

        for thread in threads:
            thread.start()
        for thread in threads:
            thread.join()

        finish_time = time.perf_counter()

        print(f"\nMejor accuracy SVM: {mejor_result_svm['accuracy']}, recall: {mejor_result_svm['recall']}, parámetros: {mejor_result_svm['params']}")
        print(f"Mejor accuracy RN: {mejor_result_rn['accuracy']}, recall: {mejor_result_rn['recall']}, parámetros: {mejor_result_rn['params']}")
        print(f"Mejor accuracy NB: {mejor_result_nb['accuracy']}, recall: {mejor_result_nb['recall']}")

        print(f"\nProgram finished in {finish_time - start_time:.2f} seconds")


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 8.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 119.4/119.4 kB 9.0 MB/s eta 0:00:00

Mejor accuracy SVM: 0.8575, recall: 0.80625, parámetros: {'C': 10, 'kernel': 'rbf', 'gamma': 0.01}
Mejor accuracy RN: 0.84, recall: 0.80625, parámetros: {'hidden_layer_sizes': (100,), 'activation': 'relu', 'solver': 'adam'}
Mejor accuracy NB: 0.7875, recall: 0.6875

Program finished in 135.77 seconds


# Clasificacion simultanea de modelos utilizando mejores parametros encontrados utilizando el dataset etiquetado conseguido de la web
(DATASET 1)


In [6]:
import numpy as np
from multiprocessing import Pool
from sklearn.svm import SVC
from sklearn.naive_bayes import GaussianNB
from sklearn.neural_network import MLPClassifier
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
import pandas as pd
import time


# utiulizar los mejores hiperparametros dados por grid search previamnet
def entrenar_modelo(tipo_modelo, x_train, y_train, x_test, y_test):
    if tipo_modelo == 'SVM':
        modelo = SVC(C=10, kernel='rbf', gamma =0.01)
    elif tipo_modelo == 'NB':
        modelo = GaussianNB()
    elif tipo_modelo == 'RN':
        modelo = MLPClassifier(hidden_layer_sizes=(50,),activation='relu', solver='adam',max_iter=300)
    else:
        raise ValueError("Tipo de modelo desconocido")

    modelo.fit(x_train, y_train)
    y_pred = modelo.predict(x_test)


    accuracy = accuracy_score(y_test, y_pred)
    return tipo_modelo, accuracy


if __name__ == '__main__':

    archivo_csv = 'embeddings_con_etiqueta.csv'
    datos = pd.read_csv(archivo_csv)

    # Mapeo de etiquetas a valores binarios
    datos['sentimiento_binario'] = datos['sentimiento'].map({'NEGATIVO': 0, 'POSITIVO': 1})


    X = datos.iloc[:, :-1].select_dtypes(include=['float', 'int']).values  # Convertir a matriz NumPy
    y = datos['sentimiento_binario'].values


    x_train, x_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42, stratify=y)

    # Asegurar que los datos no sean solo de lectura
    x_train, x_test = np.copy(x_train), np.copy(x_test)
    y_train, y_test = np.copy(y_train), np.copy(y_test)


    modelos = ['SVM', 'NB', 'RN']


    inicio = time.time()

    # Entrenamiento en paralelo
    with Pool(processes=len(modelos)) as pool:
        resultados = pool.starmap(entrenar_modelo, [(modelo, x_train, y_train, x_test, y_test) for modelo in modelos])

    # Mostramos resultados
    for modelo, accuracy in resultados:
        print(f"Modelo: {modelo}, Precisión: {accuracy:.2f}")

    print(f"Tiempo total: {time.time() - inicio:.2f} segundos")


Modelo: SVM, Precisión: 0.86
Modelo: NB, Precisión: 0.79
Modelo: RN, Precisión: 0.82
Tiempo total: 12.05 segundos
